In [1]:
from math import erfc, sqrt
from quantecon.markov.core import MarkovChain

import numpy as np
from numba import njit

Consider the AR(1) process with drift $b$,

$$
y_{t+1}=b+\rho y_{t}+u_{t+1},
$$

and the mean of the stationary distribution is $\frac{b}{(1-\rho)}$. If we divide $b$ into $\frac{1}{1-\rho}b$ and $\frac{-\rho}{1-\rho}b$ and move the first part to the l.h.s., the original AR(1) process can be written as
$$
\Leftrightarrow y_{t+1}-\frac{1}{1-\rho}b=\rho y_{t}-\frac{\rho}{1-\rho}b+u_{t+1},
$$
$$
\Leftrightarrow\tilde{y}_{t+1}=\rho\tilde{y}_{t}+u_{t+1},
$$

where $\tilde{y}_t = y_t - \frac{b}{1-\rho}$. As $\tilde{y}_t$ follows AR(1) process without drift term, and $\tilde{y}_t$ has the same standard deviation as $y_t$ which equals to $\frac{σ^2}{1 - ρ^2}$, we can use the existing tauchen code to approximate the Markov transition matrix. 

After we have the transition matrix for $\tilde{y}_t$, we can simply shift the state values by $\frac{b}{1-\rho}$ and that should be the transition matrix for $y_t$.

I made the minimal modification to the existing code to make `qe.tauchen` able to handle the non-zero drift term. `_fill_tauchen` is kept unchanged.

In [2]:
def tauchen(rho, sigma_u, b=0., m=3, n=7):

    # standard deviation of demeaned y_t
    std_y = np.sqrt(sigma_u**2 / (1 - rho**2))

    # top of discrete state space for demeaned y_t
    x_max = m * std_y

    # bottom of discrete state space for demeaned y_t
    x_min = -x_max

    # discretized state space for demeaned y_t
    x = np.linspace(x_min, x_max, n)

    step = (x_max - x_min) / (n - 1)
    half_step = 0.5 * step
    P = np.empty((n, n))

    # approximate Markov transition matrix for
    # demeaned y_t
    _fill_tauchen(x, P, n, rho, sigma_u, half_step)

    # shifts the state values by the long run mean of y_t
    mu = b / (1 - rho)

    mc = MarkovChain(P, state_values=x+mu)

    return mc

@njit
def _fill_tauchen(x, P, n, rho, sigma, half_step):
    for i in range(n):
        P[i, 0] = std_norm_cdf((x[0] - rho * x[i] + half_step) / sigma)
        P[i, n - 1] = 1 - \
            std_norm_cdf((x[n - 1] - rho * x[i] - half_step) / sigma)
        for j in range(1, n - 1):
            z = x[j] - rho * x[i]
            P[i, j] = (std_norm_cdf((z + half_step) / sigma) -
                       std_norm_cdf((z - half_step) / sigma))

@njit
def std_norm_cdf(x):
    return 0.5 * erfc(-x / sqrt(2))

To verify that the code is working, try a simple case with $ρ = 0.9, σ = 1.0,$ and $b = 1.0$.

In [3]:
ρ = 0.9
σ = 1.0
b = 1.0

The mean of the stationary distribution is

In [4]:
b / (1 - ρ)

10.000000000000002

The standard deviation of the stationay distribution is

In [5]:
np.sqrt(σ**2 / (1 - ρ**2))

2.294157338705618

Since the process is ergodic, we can approximate the transition matrix and simulate a long time series and then compare the first and the second moments.

In [6]:
mc = tauchen(ρ, σ, b=b, n=101, m=10)
simu = mc.simulate(10000000, random_state=1)
print(simu.mean())
print(simu.std())

10.00118451931713
2.314774074929692
